In [ ]:
import sys
sys.path.append('../helper')
from ls_helper import *

In [ ]:
from label_studio_sdk.label_interface import LabelInterface
from label_studio_sdk.label_interface.create import labels
from label_studio_sdk.actions import ActionsCreateRequestSelectedItemsExcluded
from dotenv import load_dotenv
import os
import logging

In [ ]:
load_dotenv()

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
PROJECT_NAME=os.getenv('PROJECT_NAME', 'GROUNDING_DINO')
RECREATE_PROJECT = int(os.getenv('RECREATE_PROJECT', '0'))

In [ ]:
logging.info('Setting up required variable values')

# Define the URL where Label Studio is accessible and the API key for your user account
LABEL_STUDIO_URL = os.getenv('LABEL_STUDIO_URL', 'http://localhost:8080')
LABEL_STUDIO_ML_BACKEND_URL = os.getenv('LABEL_STUDIO_ML_BACKEND_URL', 'http://localhost:9090')

# API key is available at the Account & Settings > Access Tokens page in Label Studio UI
API_KEY = os.getenv('API_KEY')

# Import the SDK and the client module
from label_studio_sdk.client import LabelStudio

# Connect to the Label Studio API and check the connection
ls = LabelStudio(base_url=LABEL_STUDIO_URL, api_key=API_KEY)

In [ ]:

# Create a project with the specified title and labeling configuration
project = get_or_create_project(ls, PROJECT_NAME, 
                                """
                                <View>
                                    <Style>
                                    .lsf-main-content.lsf-requesting .prompt::before { content: ' loading...'; color: #808080; }
                                    </Style>
                                    <View className="prompt">
                                        <Header value="Enter a prompt to detect objects in the image:"/>
                                    <TextArea name="prompt" toName="image" editable="true" rows="2" maxSubmissions="1" showSubmitButton="true"/>
                                    </View>
                                    <Image name="image" value="$image"/>
                                
                                    <RectangleLabels name="label" toName="image">
                                        <Label value="cats" background="yellow"/>
                                        <Label value="house" background="blue"/>
                                    </RectangleLabels>
                                </View>
                                """, 
                                RECREATE_PROJECT
                                )

In [ ]:
logging.info('Project:' + str(project))

In [ ]:
# Import sample images
if RECREATE_PROJECT:
    ls.projects.import_tasks(
        project.id,
        request=[{'image': f'https://data.heartex.net/open-images/train_0/mini/0045dd96bf73936c.jpg'},
                {'image': f'https://data.heartex.net/open-images/train_0/mini/0083d02f6ad18b38.jpg'}]
    )

In [ ]:
# Create a ML Backend
logging.info('Creating Grounding Dino ML Backend')
grounding_dino_model = get_or_create_model(ls,
    title='GROUNDING-DINO',
    description='A Grounding Dino model not needing GPU',
    url=LABEL_STUDIO_ML_BACKEND_URL,
    project_id=project.id,
    is_interactive=True
)

In [ ]:
# Retrieve predictions
logging.info('Logging the tasks')
for task in ls.tasks.list(project=project.id):
    logging.info('Predicting Task: ' + str(task))
    ls.ml.predict_interactive(
        id=grounding_dino_model.id, 
        task=task.id, 
        context={'result': [{'value': {'text': ['animal']},
             'from_name': 'prompt',
             'to_name': 'image',
             'type': 'textarea',
             'origin': 'manual'}]}
    )